<h2> Spectroscopic data reduction : spectra module

In [1]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [2]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [28]:
indir='/home/users/adijeau/Documents/UT220107'
indir='/home/holtz/red/UT211030/kosmos/'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/holtz/red/UT211030/kosmos//*{:04d}.f*.fits*
         gain:  [0.6]    rn: [ 5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [29]:
red.log()


file /home/holtz/red/UT211030/kosmos/BD28d4211.0009.fits does not have DATE-OBS


ValueError: Mismatch between number of vals and columns

A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a Data object, a numpy array, or a FITS HDU object.

In [4]:
a=red.reduce(22)
t.tv(a)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_GCPilot02_red.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


The resulting Data object has attributes: header, data, uncertainty, and bitmask, which are carried along through the processing.

In [5]:
a.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                   32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2148                                                  
NAXIS2  =                 4096                                                  
EXTEND  =                    T                                                  
OBSERVAT= 'APO'                / Per the IRAF observatory list.                 
TELESCOP= '3.5m'                                                                
INSTRUME= 'kosmos'             / Instrument name                                
LATITUDE= +3.2780361000000E+01 / Latitude of telescope base                     
LONGITUD= -1.0582041700000E+02 / Longitude of telescope base                    
UTC-TAI = -37.0                / UTC = TAI + UTC_TAI(seconds)                   
UT1-TAI = -37.11            

<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field.
<br>
First, however, we have to make the flat field, which is accomplished using the mkflat() method, which takes as input a list of frames to be used to construct the master flat field (superflat). For a spectrograph, we use the spec=True keyword which will remove the spectral signature from the combined flat.
<br>
If you specify a pyvista TV tool with the display= keyword, mkflat() will display the final flat field, then each individual input frame divided by
the final flat, so you can inspect and confirm that all input frames are similar. You will need to enter a space in the display window after each
image is displayed to proceed to the next, as prompted by the output.

In [23]:
flatims=[61,62,63,64,65]
flat=red.mkflat(flatims,spec=True,display=None,littrow=True)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0061.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0062.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0063.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0064.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0065.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....
   fitting/subtracting Littrow ghost
290 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.76407085e-03   1.72045442e+01   1.08894077e+01   1.26330078e-02
   2.3

/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


302 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.72511785e-02   3.04257466e+01  -4.21736193e+00   8.94585235e-01
   4.29086710e-06]
303 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -1.18719606e-02   2.65220152e+01  -3.63390286e+00   9.02203805e-01
  -5.80357087e-05]
304 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
305 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -9.29719046e-03   2.32778218e+01  -1.06483547e+00   9.11044096e-01
   9.17705655e-05]
306 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.98141871e-02   2.72639013e+00   7.27002412e+00   9.49891734e-01
  -1.63318440e-03]
307 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2

25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  6.17365469e-02   3.96288759e+00   1.55640226e+01   8.84206992e-01
   1.45516009e-03]
362 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.76556927e-02   1.01735652e+01   8.08241992e+00   9.36078229e-01
   2.38470677e-04]
363 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.53977268e-02   1.07565500e+01   1.07912532e+01   9.28594139e-01
   1.78967957e-04]
364 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.80135652e-02   1.11750709e+01   8.21358973e+00   9.37024793e-01
   6.66019347e-05]
365 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.79063289e-02   1.21086834e+01   7.82825667e+00   9.38606973e-01
  -5.63825503e-05]
366 2000 2025
25
[0.05, 

420 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.72830535e-02   9.29598815e+00   4.94297022e+00   9.37994022e-01
   3.61775733e-04]
421 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.81864223e-02   9.22149304e+00   5.66730967e+00   9.35974397e-01
   4.09627119e-04]
422 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.68517511e-02   9.48282876e+00   5.19562052e+00   9.37990659e-01
   3.62092615e-04]
423 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.76615465e-02   9.01026979e+00   5.53225033e+00   9.36169850e-01
   4.40432037e-04]
424 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.79247451e-02   9.12508513e+00   5.71225166e+00   9.35293544e-01
   4.80530346e-04]
425 2000 2

...

1410 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.17539775e-02   8.35834665e+00   5.90899950e+00   8.85527438e-01
   3.53996671e-04]
1411 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.89918957e-02   8.62471258e+00   4.97140571e+00   8.89897969e-01
   2.11650208e-04]
1412 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.58432182e-02   7.74216188e+00   6.71103204e+00   8.80108494e-01
   5.41011056e-04]
1413 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.23158800e-02   8.06592110e+00   5.92640555e+00   8.84920237e-01
   3.89865434e-04]
1414 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.41789550e-02   7.80215749e+00   6.33839962e+00   8.81972000e-01
   4.92177318e-04]
1415 

[  1.59153480e-02   8.39351098e+00  -3.96329090e+00   9.05508608e-01
  -1.94997986e-04]
1468 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.65571041e-02   8.12082621e+00   4.04694457e+00   9.03563582e-01
  -9.67019412e-05]
1469 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.65831519e-02   8.16692394e+00   4.06689878e+00   9.03480001e-01
  -7.14351471e-05]
1470 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.66439033e-02   8.31576336e+00   3.74568643e+00   9.04442402e-01
  -9.97312481e-05]
1471 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.56777583e-02   8.38342983e+00   3.48693483e+00   9.06343932e-01
  -1.79996091e-04]
1472 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.

[ -1.65852285e-02  -1.25737728e-01  -3.10121919e+00   9.31415779e-01
  -1.30749542e-03]
1513 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -1.97061766e-02  -1.96168485e+00  -3.90792800e+00   9.32080841e-01
  -1.36021003e-03]
1514 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.29782009e-02  -2.30778603e+00   4.22239427e+00   9.34860693e-01
  -1.51416468e-03]
1515 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.42240520e-02   8.45757605e+00  -4.73669954e+00   9.13818167e-01
  -5.59745986e-04]
1516 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.38051804e-02   9.13027776e+00  -4.15730753e+00   9.16215056e-01
  -6.98067038e-04]
1517 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.

1580 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.55787132e-02   8.25964801e+00   4.19760956e+00   9.11381028e-01
  -3.24062444e-04]
1581 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.54768538e-02   8.44615398e+00   4.15037525e+00   9.11720025e-01
  -3.48907518e-04]
1582 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[ -2.12019130e-02  -6.44910942e-01   3.72067236e+00   9.35550988e-01
  -1.46641676e-03]
1583 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.36775193e-02   9.08311345e+00  -3.81262080e+00   9.15539926e-01
  -5.42119218e-04]
1584 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.32057694e-02   9.15337513e+00  -3.64330962e+00   9.16372711e-01
  -5.50283222e-04]
1585 

1629 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
1630 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  2.47505155e-02   7.04570685e+00   8.51500069e+00   8.90485275e-01
  -2.01725430e-04]
1631 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.73466703e-02   8.29432815e+00   6.92070423e+00   9.00194901e-01
  -4.73194534e-04]
1632 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.30974214e-02   9.01854691e+00   5.83020883e+00   9.06155373e-01
  -6.30162742e-04]
1633 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 12.5, 2.0, 0.001))
[  1.30228735e-02   9.48821006e+00   5.91249586e+00   9.07779687e-01
  -6.70957847e-04]
1634 2000 2025
25
[0.05, 12.5, 6.25, 1.0, 0.0]
((0.0, 6.25, 0.0, 0.0, -0.001), (0.2, 18.75, 1

In [22]:
t.tv(flat)

Read, overscan subtract, flat field, and display a star spectral image

In [ ]:
star=red.reduce(2,flat=flat)
t.tv(star,max=1000) 

Here is an example of attempting cosmic ray rejection. CR rejection can be challenging; the main concern is that the cosmic ray rejection flags object pixels as cosmic rays! Users should be very careful about this. The Reducer object provides two algorithms for CR-rejection. If crbox='lacosmic", then the LA Cosmic routine of van Dokkum, as implemented
in the astroscrappy package, is used. If crbox=[nrow,ncol], then the image
is median-filtered with the specfied box shape, and pixels above nsig (default=5) times the uncertainty are replaced with the median. For spectroscopy, beware: a box aligned with the wavelength direction will
avoid flagging pixels in the object, but may flag pixels in sky lines,
while the reverse is true for a box aligned along the slit.

In [ ]:
crstar=red.crrej(star,crbox=[9,1],display=t)

<h4> Tracing and extraction

We want to extract the spectrum, i.e. from the 2D image to a 1D spectrum. Start with a trace previously determined for KOSMOS. The Trace object sets the order of the polynomial used to fit the trace (degree attribute), the length of the slit (rows attribute), the range of pixels along the slit to be used to automatically find an object, etc.

In [ ]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
vars(trace) 

We can use the existing trace to retrace a new spectrum. This is done by cross-correlating the reference spectrum with a spatial cut across the input spectrum (at sc0) to find the shift, then using the old model to start the trace from that starting position. The functional form of the old model is preserved:

In [ ]:
trace.retrace(star,display=t)

Alternatively, one could use the find() method to determine the shift between the stored trace and your frame automatically with cross correlation, then just go straight to extraction using the shape of the stored trace; this would be applicable if your object is too faint, or doesn't have sufficient continuum, to trace. If you want to mark the location of the object manually, use the inter=True keyword (along with display=) in find(). 


In [ ]:
trace.model[0](2048)
trace.find(star,inter=True,display=t)
print(trace.pix0)

You can also just find peak(s) in the spatial direction using findpeak(), then trace them:

In [ ]:
peaks,fibers=trace.findpeak(star,thresh=5)
print(peaks)
print(fibers)
                            
trace.trace(star,peaks,display=t)

The model is now modified for the input spectrum:

In [ ]:
trace.model[0](2048)

Her are some various alternatives for finding traces. If you needed to create Trace from scratch, instantiate one with rows=, lags=, sc= (transpose= determines whether inputimage needs to be transposed to get wavelength changing along columns). 
<p>
    Note the skip= keyword which can be used when tracing to speed things up by only computing centroids every skip pixels, taking a median around these pixels of width skip pixels. The default is skip=10

In [ ]:
trace=spectra.Trace(sc0=2048,lags=range(-300,300),
                    rows=[550,1450],transpose=red.transpose)
vars(trace)
srow=[955]   # list to allow for multiple spectra on an image, mnaually set
srow,fibers=trace.findpeak(star)  # alternatively, find peak(s)
# trace.find(star) will find the highest peak by cross-correlation
# trace.find(star,inter=True,display=t)  will let you mark a trace location
starec=trace.trace(star,srow,display=t,skip=100) 
vars(trace)

Extraction is done using the extract() method of the Trace object, currently just with simple boxcar extraction. Use rad= to specify window radius. Use back=[[b1,b2],[b3,b4]] to subtract background as determined from one or more background windows (note argument should be list of 2-element lists, giving start and end pixel of each background window).

In [ ]:
starec=trace.extract(star,display=t,rad=25)
print(starec.shape)

extract() returns a pyvista Data object, with extracted spectrum, and uncertainty. Note that this is a 2D array to accomodate multiple objects/traces, even if there is only a single object.

In [ ]:
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec.data[0]/starec.uncertainty.array[0])

Background can be subtracted during extraction using the back= keyword, which takes a list of 2-element lists, given the relative location, in pixels, of the background window(s) (relative to the location of the trace):

In [ ]:
starec2=trace.extract(star,rad=25,back=[[50,75],[-75,-50]],display=t)
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec2.data[0])
plt.show()

<h4> Wavelength calibration

Now let's turn to wavelength calibration, i.e. getting a function that gives the wavelength as a function of pixel. We'll solve for this using arc frames, here taken with each lamp separately, so sum the three exposures

In [ ]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([46,47,48])
t.clear()
t.tv(arcs)

In [ ]:
arcec=trace.extract(arcs,display=None,rad=20)

Wavelength calibration first stars with identifying lines. This is much easier if one can work from a previous solution. Here we start by reading a previous solution into a pyvista WaveCal object.

In [ ]:
wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
vars(wav)

In [ ]:
wav.dispersion()

With a previous solution loaded, the identify routine will cross correlate the input spectrum with the previous solution, then attempt to identify the lines from the previous solution at the shifted pixel position from the previous solution. Finally, it does a fit.

The identify() method fits Gaussians to the lines to get their position, and gets FWHM in the process. rad= keyword specifies number of pixels on either side of line to use in fit. If you add the file= keyword, then the routine will try to identify all of the lines from the reference file; this might be useful if you wanted to extend the wavelength range of the lines from the initial object, e.g. if you were working at a different grating tilt or slit location. If you did that, you might want to save your WaveCal object after cleaning the line list for good lines, so you could use that as a starting guess for other data taken at a similar wavelength setting.

In [ ]:
wav.identify(arcec,plot=True,rad=10) #,file='henearkr.dat')

Note that once you remove lines, they are removed for subsequent uses of the WaveCal object.

If you want to refit with existing line positions, e.g. to try a different model, you can just call fit().

In [ ]:
wav.fit(degree=5)  

Show the FWHM of the lines as a function of wavelength

In [ ]:
wav.fwhm
t.plotax2.cla()
gd=np.where(wav.weights > 0)[0]
t.plotax2.plot(wav.waves[gd],np.abs(wav.fwhm[gd]),'ro')

OK, now use the wavelength solution to get wavelength as a function of pixel. We can save that in the wave attribute of our Data object.

In [ ]:
wav.add_wave(starec)
plt.figure()
plt.plot(starec.wave[0],starec.data[0])

Here is a way to get the inverse wavelength solution, i.e. pixels as a function of wavelength, using a cubic spline fit to the wavelength solution.

In [ ]:
# get inverse relation, i.e. pixels as f(wavelength)
pix=np.arange(4096)
from scipy.interpolate import CubicSpline
wav2pix=CubicSpline(np.flip(starec.wave[0]),np.flip(pix))
print('wavelength 6563 expected at pixel: ',wav2pix(6463))

Next cell demonstrates resampling of spectrum onto a uniform wavelength grid. This may not be desired, as it leads to correlated uncertainties between pixels! But it would be needed if, e.g., you wanted to cross-correlate spectra to find velocities, in which case you would want a spectrum with constant dispersion in log(lambda).

In [ ]:
plt.figure()
plt.clf()
# original data with wavelength solution
plt.plot(starec.wave[0],starec.data[0])

# set desired new wavelength scale:
wnew=10**np.arange(3.5,4.0,5.5e-6)
# resampled data on constant log(lambda) grid
plt.plot(wnew,wav.scomb(starec,wnew).data)

<h4> Adjusting wavelength solution for flexure

In [ ]:
objec=trace.extract(red.reduce(22),display=None,rad=20)
wav.add_wave(objec)

In [ ]:
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

The skyline() method will addjust the current wavelength solution based on finding sky emission lines. The default behavior is to adopt the current wavelength solution but allow only the 0th order term to shift. If you specify linear=True, then the 1st order term is also allowed to vary; beware that you must have multiple sky lines for this to be effective! Otherwise, skyline() takes most of the same keywords as identify()
<br>
Here, I first copy the WaveCal object so we can compare the original to the corrected solution.

In [ ]:
import copy
swav=copy.deepcopy(wav)
swav.skyline( object, thresh=50)

See how the model has been modified compared to the original:

In [ ]:
print(wav.model)
print(swav.model)

<h4> Flux calibration

Instantiate a FluxCal object. You can set it up to use a polynomial fit to the response curve by specifying the polynomial degree with the degree= keyword. If you set degree=-1, the response curve will be determined by a median of the individual response curves, optionally smoothed with a median filter over wavelength (see response() method below).

In [ ]:
flx=spectra.FluxCal(degree=-1)
polyflx=spectra.FluxCal(degree=5)

Load in several extracted spectra of flux standards, here all of Feige 34. The standard star spectrum is given using the file= keyword, where the input file should have labelled columns wave, flux, and bin. Alternatively, you can pass an astropy Table with (at least) these three columns, using the stdflux= keyword

In [ ]:
for im in [2,3, 38,39] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    wav.add_wave(starec)
    flx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')
    polyflx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')

Solve for the median response curve, taking a median filter over this median.

In [ ]:
flx.response(plot=True,medfilt=200)
polyflx.response(plot=True)

Apply the response curve to an object. Note that, using Data objects, the S/N is preserved!

In [ ]:
fig,ax=plots.multi(1,2,hspace=0.001)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])
flx.correct(starec,starec.wave)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])

<h3> longslit extraction and wavelength calibration

For extended objects, and perhaps for sky subtraction, we might want to work along the slit. The wavelength solution varies along the slit (line curvature), usually with more than just an offset.

Start by working along the slit to identify lines

In [ ]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arc2d=trace.extract2d(arcs)
t=tv.TV()
t.tv(arc2d)

In [ ]:
from pyvista import image
wav=spectra.WaveCal(file='KOSMOS/KOSMOS_red_waves.fits')
image.smooth(arc2d,[5,1])
t.clear()
t.tv(arc2d)
wav.identify(arc2d, rad=10,display=None, plot=True,
              nskip=20,lags=np.arange(-10,10))
wav.add_wave(arc2d)

In [ ]:
wav.plot()


OK, use the solution to make a wavelength map

In [ ]:
t.clear()
t.tv(arc2d.wave)

Subtract out the central wavelength solution to see how the solution varies with row

In [ ]:
dw=arc2d.wave-arc2d.wave[450]
t.tv(dw)  

Now do a 2d extraction of an object, and attach the wavelengths

In [ ]:
obj=red.reduce(22)
t.tv(obj)

In [ ]:

objcr=red.crrej(obj,crbox=[1,9],nsig=5)
t.tv(obj)
t.tv(objcr)
objcr2=red.crrej(obj,crbox=[1,9],nsig=[5,2])
t.tv(objcr2)

In [ ]:
t.tv(objcr.bitmask)
from pyvista import image
t.tv(image.smooth(objcr.bitmask,[3,3]))

In [ ]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
obj=red.reduce(22,crbox=[1,9])
t.tv(obj)
obj2d=trace.extract2d(obj)
wav.add_wave(obj2d)
t.tv(obj2d)
star=red.reduce(38)
star2d=trace.extract2d(star)
wav.add_wave(star2d)

In [ ]:
t.tv(obj.bitmask)
t.tv(obj2d.bitmask)

Here we rectify the image to have a constant wavelength scale (in log lambda). We choose the new scale based on the starting and ending wavelengths in the original image, and resample to get the same number of pixels.

If you preferred not to resample your object (or at least minimize the ressampling), you could resample the image to the wavelength array at the location of your object

In [ ]:
wlim=(arc2d.wave[450,0],arc2d.wave[450,-1])
wnew=10.**np.linspace(np.log10(wlim[1]),np.log10(wlim[0]),4095)
print(wnew)
arc2d_rect=wav.correct(arc2d,wnew)
t.tv(arc2d)
t.tv(arc2d_rect)

In [ ]:
t.tv(obj2d)
obj2d_rect=wav.correct(obj2d,wnew)
star2d_rect=wav.correct(star2d,wnew)
t.tv(obj2d_rect)


OK, now we can extract in the wavelength rectified image for better sky subtraction. Create a new trace object with the rows= attribute to give the new extent of the image in rows, since our previous extraction limited the extraction to the length of the slit, as set by the rows attribute in the original trace object.

In [ ]:
rows=[0,star2d_rect.shape[0]]
trace=spectra.Trace(sc0=int(star2d_rect.shape[1]/2.),rows=rows)
t.clear()
rows,fibers=trace.findpeak(star2d_rect)
print('rows: ',rows)
trace.trace(star2d_rect,rows,display=t)
ext=trace.extract(obj2d_rect,display=t,rad=10)
ext.add_wave(wnew)
ext_sub=trace.extract(obj2d_rect,rad=10,back=[[20,30]],display=t)
ext_sub.add_wave(wnew)

In [ ]:
plt.figure()
plt.plot(ext.wave,ext.data[0])
plt.plot(ext.wave,ext_sub.data[0])

CR rejection via multiple image stacking

<h4> KOSMOS blue channel

In [ ]:
bwav=spectra.WaveCal('KOSMOS/KOSMOS_blue_waves.fits')
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arcs=red.sum([49,50,51])
arcec=trace.extract(arcs)
bwav.identify(arcec,plot=True)


In [ ]:
objec=trace.extract(red.reduce(26),display=None,rad=20)
bwav.add_wave(objec)
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

swav=copy.copy(bwav)
swav.model.fixed['c0']=False
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(bwav.model)
print(swav.model)

In [ ]:
flx=spectra.FluxCal(degree=5)
for im in [34,35,36] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    bwav.add_wave(starec)
    flx.addstar(starec[0],wave[0],file='flux/okestan/ffeige34.dat')

In [ ]:
flx.response(plot=True)

In [ ]:
plt.figure()
flx.correct(starec,starec.wave[0])
plt.plot(starec.wave[0],starec.data[0])